In [1]:
# Basic Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets, models

# Scikit-learn
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from PIL import Image


In [2]:
# Dataset root folder
DATA_ROOT = "./GTSRB"
TRAIN_DIR = os.path.join(DATA_ROOT, "Train")
TEST_DIR  = os.path.join(DATA_ROOT, "Test")

TRAIN_CSV = os.path.join(DATA_ROOT, "Train.csv")
TEST_CSV  = os.path.join(DATA_ROOT, "Test.csv")

# Device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [3]:
# Data augmentations for training
train_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# No augmentation for test set
test_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [4]:
train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

num_classes = len(train_dataset.classes)
print("Number of classes:", num_classes)


Number of classes: 43


In [5]:
class TrafficSignTestDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.root_dir, os.path.basename(row["Path"]))
        image = Image.open(img_path).convert("RGB")
        label = int(row["ClassId"])
        if self.transform:
            image = self.transform(image)
        return image, label

test_dataset = TrafficSignTestDataset(TEST_CSV, TEST_DIR, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
print("Test samples:", len(test_dataset))


Test samples: 12630


In [6]:
class TrafficSignCNN(nn.Module):
    def __init__(self, num_classes):
        super(TrafficSignCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

custom_cnn = TrafficSignCNN(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer_cnn = optim.Adam(custom_cnn.parameters(), lr=0.001)


In [7]:
EPOCHS = 10
for epoch in range(EPOCHS):
    custom_cnn.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer_cnn.zero_grad()
        outputs = custom_cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_cnn.step()
        running_loss += loss.item()
    print(f"[Custom CNN] Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss/len(train_loader):.4f}")


[Custom CNN] Epoch [1/10], Loss: 1.0414
[Custom CNN] Epoch [2/10], Loss: 0.2691
[Custom CNN] Epoch [3/10], Loss: 0.1627
[Custom CNN] Epoch [4/10], Loss: 0.1209
[Custom CNN] Epoch [5/10], Loss: 0.0916
[Custom CNN] Epoch [6/10], Loss: 0.0781
[Custom CNN] Epoch [7/10], Loss: 0.0650
[Custom CNN] Epoch [8/10], Loss: 0.0576
[Custom CNN] Epoch [9/10], Loss: 0.0525
[Custom CNN] Epoch [10/10], Loss: 0.0488


In [8]:
custom_cnn.eval()
y_true_cnn, y_pred_cnn = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = custom_cnn(images)
        _, preds = torch.max(outputs, 1)
        y_true_cnn.extend(labels.cpu().numpy())
        y_pred_cnn.extend(preds.cpu().numpy())

print("Classification Report (Custom CNN):\n")
print(classification_report(y_true_cnn, y_pred_cnn, digits=4))


Classification Report (Custom CNN):

              precision    recall  f1-score   support

           0     0.5795    0.8500    0.6892        60
           1     0.9354    0.8250    0.8768       720
           2     0.0000    0.0000    0.0000       750
           3     0.0024    0.0022    0.0023       450
           4     0.0000    0.0000    0.0000       660
           5     0.0000    0.0000    0.0000       630
           6     0.0000    0.0000    0.0000       150
           7     0.0000    0.0000    0.0000       450
           8     0.0000    0.0000    0.0000       450
           9     0.0000    0.0000    0.0000       480
          10     0.0037    0.0015    0.0022       660
          11     0.0000    0.0000    0.0000       420
          12     0.0000    0.0000    0.0000       690
          13     0.0000    0.0000    0.0000       720
          14     0.0000    0.0000    0.0000       270
          15     0.0000    0.0000    0.0000       210
          16     0.0000    0.0000    0.0000 

# Pretrained MobileNetV2

In [9]:
mobilenet = models.mobilenet_v2(pretrained=True)
for param in mobilenet.features.parameters():
    param.requires_grad = False

mobilenet.classifier[1] = nn.Linear(1280, num_classes)
mobilenet = mobilenet.to(device)

optimizer_mobilenet = optim.Adam(mobilenet.classifier.parameters(), lr=0.001)


/home/mohsin/miniconda/envs/dl_env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mohsin/miniconda/envs/dl_env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Train MonileNetV2

In [10]:
EPOCHS = 5
for epoch in range(EPOCHS):
    mobilenet.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer_mobilenet.zero_grad()
        outputs = mobilenet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_mobilenet.step()
        running_loss += loss.item()
    print(f"[MobileNetV2] Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss/len(train_loader):.4f}")


[MobileNetV2] Epoch [1/5], Loss: 1.7907
[MobileNetV2] Epoch [2/5], Loss: 1.4365
[MobileNetV2] Epoch [3/5], Loss: 1.3785
[MobileNetV2] Epoch [4/5], Loss: 1.3332
[MobileNetV2] Epoch [5/5], Loss: 1.3173


# Evaluate MobileNetV2

In [11]:
mobilenet.eval()
y_true_mobilenet, y_pred_mobilenet = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = mobilenet(images)
        _, preds = torch.max(outputs, 1)
        y_true_mobilenet.extend(labels.cpu().numpy())
        y_pred_mobilenet.extend(preds.cpu().numpy())

print("Classification Report (MobileNetV2):\n")
print(classification_report(y_true_mobilenet, y_pred_mobilenet, digits=4))


Classification Report (MobileNetV2):

              precision    recall  f1-score   support

           0     0.4615    0.2000    0.2791        60
           1     0.5386    0.5042    0.5208       720
           2     0.0125    0.0093    0.0107       750
           3     0.0000    0.0000    0.0000       450
           4     0.0142    0.0167    0.0153       660
           5     0.0000    0.0000    0.0000       630
           6     0.0000    0.0000    0.0000       150
           7     0.0047    0.0022    0.0030       450
           8     0.0061    0.0022    0.0033       450
           9     0.0355    0.0271    0.0307       480
          10     0.0000    0.0000    0.0000       660
          11     0.2500    0.0024    0.0047       420
          12     0.0029    0.0029    0.0029       690
          13     0.0185    0.0014    0.0026       720
          14     0.0000    0.0000    0.0000       270
          15     0.0000    0.0000    0.0000       210
          16     0.0000    0.0000    0.0000

# Comparison Table

In [12]:
custom_cnn_accuracy = accuracy_score(y_true_cnn, y_pred_cnn)
mobilenet_accuracy  = accuracy_score(y_true_mobilenet, y_pred_mobilenet)

results = {
    "Model": ["Custom CNN", "MobileNetV2"],
    "Test Accuracy": [custom_cnn_accuracy, mobilenet_accuracy]
}
df = pd.DataFrame(results)
print("\nComparison Table:\n", df)



Comparison Table:
          Model  Test Accuracy
0   Custom CNN       0.058591
1  MobileNetV2       0.043785
